### Learning Tensorboard Basics

Following tutorial from https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/4_Utils/tensorboard_advanced.ipynb

In [1]:
import tensorflow as tf

#import mnist dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

/Users/Nina/Documents/Streamba/tensorflow/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1
logs_path = '/logs/tensorboard_basics/'

# Network Parameters
n_hidden_1 = 256 # 1st layer n. of features
n_hidden_2 = 256 # 2nd layer n. of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST classes (0-9 digits)

### Phase 1: assemble your graph

1. Import data (either with tf.data or with placeholders)
2. Define the weights
3. Define the inference model
4. Define loss function
5. Define optimizer

**placeholders**: variable to which data will be assigned later on. Allows for building graphs without having the data.

```placehoder(type, shape, name)```

In [3]:
# tf Graph Input
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition => 10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

### creating the model (step 3)

try making it as pythonic as possible. Good modularity

Create a basic multilayer perceptron with 2 hidden layers and relu activation funciton.

**tf.summary**: way to condense information that can then be visualised in the Tensorboard. Summaries can be saved to file using **_tf.summary.FileWriter( some-directory, sess.graph)_**

In [5]:
# Store layers weight & bias
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='W1'),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='W2'),
    'w3': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='W3')
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'b3': tf.Variable(tf.random_normal([n_classes]), name='b3')
}

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1']) 
    layer_1 = tf.nn.relu(layer_1)
    # Create a summary to visualize the first layer ReLU activation
    tf.summary.histogram("relu1", layer_1) #summary ops
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Create another summary to visualize the second layer ReLU activation
    tf.summary.histogram("relu2", layer_2)
    # Output layer
    out_layer = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    return out_layer



**NAME SHARING**: groups ops into *scopes*. Aids with visualizing in tensorboard

Also follow by defining **loss** and **optimization** functions (steps 4 and 5)

In [ ]:
with tf.name_scope('Model'):
    percep = multilayer_perceptron(x, weights, biases)
    
with tf.name_scope('Loss'):
    #create loss function
    #softmax cross entropy
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=percep, labels=y))

with tf.name_scope('Optimizer'):
    # Op to calculate variable gradient
    grads = tf.gradients(loss, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    
    #create optimizer - STOCHASTIC GRADIENT DESCENT
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).apply_gradients(grads_and_vars=grads)

    
with tf.name_scope('Accuracy'):
    acc = tf.equal(tf.argmax(percep, 1), tf.argmax(y,))
    